In [3]:
import pandas as pd
import numpy as np
from untrade.client import Client

In [8]:
import pandas as pd
import numpy as np
!rm -rf processed_data.csv

def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL) and take-profit (TP) logic based on the new strategy.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC and volume data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    df = df.copy()
    
    # Calculate moving averages
    df['MA_13'] = df['close'].rolling(window=13).mean()
    df['MA_15'] = df['close'].rolling(window=15).mean()
    df['MA_200'] = df['close'].rolling(window=200).mean()
    
    # Calculate Bollinger Bands (20-period SMA with 2 standard deviations)
    df['BB_Mid'] = df['close'].rolling(window=20).mean()
    df['BB_Upper'] = df['BB_Mid'] + (2 * df['close'].rolling(window=20).std())
    df['BB_Lower'] = df['BB_Mid'] - (2 * df['close'].rolling(window=20).std())
    
    # Calculate average volume over the last 26 periods
    df['Avg_Volume'] = df['volume'].rolling(window=26).mean()
    
    # Initialize trade columns
    df['signals'] = 0  # Buy (1), Sell (-1), Square-off (0)
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', 'square-off')
    
    prev_trade = 0  # Track previous trade state
    # Loop through data to calculate signals and update trade states
    for i in range(200, len(df)):
        curr_close = df['close'].iloc[i]
        curr_low = df['low'].iloc[i]
        curr_high = df['high'].iloc[i]
        curr_vol = df['volume'].iloc[i]
        avg_vol = df['Avg_Volume'].iloc[i]
        # prev_trade = df['trade'].iloc[i-1]
        
        # Check if volume condition is met
        vol_condition = curr_vol >= 1.3 * avg_vol
        
        # Long condition
        if df['MA_13'].iloc[i] > df['MA_15'].iloc[i] and curr_close > df['MA_200'].iloc[i] and vol_condition:
            delta = curr_close - curr_low
            df.loc[df.index[i], 'SL'] = curr_low - (0.1 * delta)
            df.loc[df.index[i], 'TP'] = curr_close + 1.5*delta
            if prev_trade == -1:  # Square-off short trade
                df.loc[df.index[i], 'signals'] = -2
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
            else:  # Open long trade
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
            prev_trade = 1
        
        # Short condition
        elif df['MA_13'].iloc[i] < df['MA_15'].iloc[i] and curr_close < df['MA_200'].iloc[i] and vol_condition:
            delta = curr_high - curr_close
            df.loc[df.index[i], 'SL'] = curr_high + (0.1 * delta)
            df.loc[df.index[i], 'TP'] = curr_close - 1.5*delta
            if prev_trade == 1:  # Square-off long trade
                df.loc[df.index[i], 'signals'] = 2
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
            else:  # Open short trade
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
            prev_trade = -1
        # else:
        #     df.loc[df.index[i], 'trade'] = prev_trade  # Maintain trade state
    
    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 'SL', 'TP', 'MA_13', 'MA_15', 'MA_200', 'BB_Upper', 'BB_Lower']]
    
    return df



def strat(data):
    """
    Filter out consecutive duplicate signals to ensure no repetitive trades.

    Parameters:
    - data (DataFrame): Dataframe with trade signals.

    Returns:
    - data (DataFrame): Dataframe with filtered signals.
    """
    data = data.copy()
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value

    data["signals"] = signal

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low',
                'close', 'volume', 'signals', 'trade_type']]
    
    data["leverage"] = 4
    # data["position"] = 100
    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        leverage=4 # Set leverage for backtest
    )

    return result

# Load ETH data


data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 131,
      "Leverage Applied": 4.0,
      "Winning Trades": 44,
      "Losing Trades": 87,
      "No. of Long Trades": 65,
      "No. of Short Trades": 66,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 33.587786,
      "Winning Streak": 3,
      "Losing Streak": 8,
      "Gross Profit": 14335.72905,
      "Net Profit": 13549.72905,
      "Average Profit": 103.433046,
      "Maximum Drawdown(%)": 28.311336,
      "Average Drawdown(%)": 6.690512,
      "Largest Win": 5932.841941,
      "Average Win": 657.337196,
      "Largest Loss": -1198.024777,
      "Average Loss": -176.702386,
      "Maximum Holding Time": "93 days 3:59:59",
      "Average Holding Time": "9 days 20:7:19",
      "Maximum Adverse Excursion": 48.841067